# Model with Julia

In [6]:
using CSV, DataFrames, Statistics, Random, ParallelKMeans, Gurobi, JuMP

In [42]:
df = CSV.read("data/pred_prev_t.csv", DataFrame)
df

Row,Column1,State,Date,population,confirmed,deaths,recovered,people_hospitalized,hospitalization_rate,incident_rate,mortality_rate,testing_rate,TestsReported,Neighbors,Value
,Int64,String31,Date,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,String31?,String?
1,0,Alabama,2020-04-12,0.111072,0.00462367,0.00282186,0.0,0.00485583,0.318442,0.014383,0.267967,0.00795396,2165,missing,missing
2,1,Alaska,2020-04-12,0.00392428,2.80818e-6,0.000242741,0.00013005,0.000344464,0.296104,0.00472572,0.301848,0.025844,262,missing,missing
3,2,Arizona,2020-04-12,0.172087,0.00459418,0.0034894,0.0,0.0,0.0,0.00572661,0.333676,0.0103453,1119,missing,missing
4,3,Arkansas,2020-04-12,0.0626465,0.00141813,0.000819249,0.000723154,0.00144452,0.263896,0.00597367,0.216632,0.0140518,625,missing,missing
5,4,California,2020-04-12,1.0,0.0316271,0.0194192,0.0,0.0581588,0.596364,0.00872928,0.288501,0.00846209,10816,missing,missing
6,5,Colorado,2020-04-12,0.133047,0.00988058,0.008769,0.0,0.0152897,0.489091,0.0311543,0.406571,0.0110912,10,missing,missing
7,6,Connecticut,2020-04-12,0.0767085,0.0165191,0.0168098,0.0,0.0183788,0.356883,0.097232,0.472279,0.0220298,2703,missing,missing
8,7,Delaware,2020-04-12,0.0101456,0.00190254,0.00106199,0.000376355,0.00211123,0.303636,0.0431713,0.220739,0.0217074,496,missing,missing
9,8,District of Columbia,2020-04-12,0.00326172,0.00225356,0.00151713,0.00097143,0.0,0.0,0.074465,0.274127,0.0291394,547,missing,missing


In [43]:
df_no_missing = dropmissing(df);

In [44]:
env = Gurobi.Env();
function optimisation_cost(y, solver_output=0)
    n = length(y)
    
    # Build model
    model = Model(()-> Gurobi.Optimizer(env))
    set_optimizer_attribute(model, "OutputFlag", solver_output) 
    
    # Variables
    @variable(model, z >= 0)
    @variable(model, s[i in 1:n])
    @variable(model, t[i in 1:n] >= 0)

    # Constraint
    @constraint(model, [i in 1:n], s[i] >= - y[i])
    @constraint(model, [i in 1:n], s[i] >= - z)
    @constraint(model, [i in 1:n], t[i] >= 0)
    @constraint(model, [i in 1:n], t[i] >= z - y[i])
    
    # Objective
    @objective(model, Min,  sum(2*z + 45*s[i] + 10*t[i] for i in 1:n))
    
    # Optimize
    optimize!(model)
    
    # Return estimated betas
        
    return (value.(z))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-23


optimisation_cost (generic function with 2 methods)

In [45]:
to_pred = [eval(Meta.parse(x)) for x in df_no_missing[:, "Value"]];

In [46]:
to_pred[1]

4-element Vector{Int64}:
 2165
  808
  625
 3390

In [47]:
total_sum = 0.0
for x in to_pred
    total_sum += optimisation_cost(x)
end
total_sum

9.7503605e7